In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.2.1
SparkUI available at http://10.128.0.2:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.4-a86d9a7bd0f3
LOGGING: writing to /home/hail/hail-20181206-1528-0.2.4-a86d9a7bd0f3.log


### Annotating genotyped datasets

##### Script: annot_genotype.py

cluster submit t1 

import hail as hl

an_gen = hl.read_matrix_table('gs://phenotype_31063/hail/ukb31063.genotype.mt')
an_gen = hl.vep(an_gen, 'gs://hail-common/vep/vep/vep85-loftee-gcloud.json')
an_gen.describe()
#write to a bucket
an_gen.write('gs://rec_project/genotype_annotations.mt', overwrite=True)


### Use vep output as input to script calculate_rare_scores (provided by Andrea Ganna). Script simplifies vep output and adds variant_qc.

In [ ]:
vds = hl.read_matrix_table('gs://rec_project/genotype_annotations.mt')
#vds.rows().show()

In [ ]:
import hail as hl
import hail.expr.aggregators as agg
import hail.methods
import pandas as pd
from typing import *
import random


# Note that this is the current as of v81 with some included for backwards compatibility (VEP <= 75)
CSQ_CODING_HIGH_IMPACT = [
    "transcript_ablation",
    "splice_acceptor_variant",
    "splice_donor_variant",
    "stop_gained",
    "frameshift_variant",
    "stop_lost"]

CSQ_CODING_MEDIUM_IMPACT = [
    "start_lost",  # new in v81
    "initiator_codon_variant",  # deprecated
    "transcript_amplification",
    "inframe_insertion",
    "inframe_deletion",
    "missense_variant",
    "protein_altering_variant",  # new in v79
    "splice_region_variant"
]

CSQ_CODING_LOW_IMPACT = [
    "incomplete_terminal_codon_variant",
    "start_retained_variant",  # new in v92
    "stop_retained_variant",
    "synonymous_variant",
    "coding_sequence_variant"]

CSQ_NON_CODING = [
    "mature_miRNA_variant",
    "5_prime_UTR_variant",
    "3_prime_UTR_variant",
    "non_coding_transcript_exon_variant",
    "non_coding_exon_variant",  # deprecated
    "intron_variant",
    "NMD_transcript_variant",
    "non_coding_transcript_variant",
    "nc_transcript_variant",  # deprecated
    "upstream_gene_variant",
    "downstream_gene_variant",
    "TFBS_ablation",
    "TFBS_amplification",
    "TF_binding_site_variant",
    "regulatory_region_ablation",
    "regulatory_region_amplification",
    "feature_elongation",
    "regulatory_region_variant",
    "feature_truncation",
    "intergenic_variant"
]

CSQ_ORDER = CSQ_CODING_HIGH_IMPACT + CSQ_CODING_MEDIUM_IMPACT + CSQ_CODING_LOW_IMPACT + CSQ_NON_CODING

def filter_vep_to_canonical_transcripts(mt: Union[hl.MatrixTable, hl.Table],
                                        vep_root: str = 'vep') -> Union[hl.MatrixTable, hl.Table]:
    canonical = mt[vep_root].transcript_consequences.filter(lambda csq: csq.canonical == 1)
    vep_data = mt[vep_root].annotate(transcript_consequences=canonical)
    return mt.annotate_rows(**{vep_root: vep_data}) if isinstance(mt, hl.MatrixTable) else mt.annotate(**{vep_root: vep_data})

def add_most_severe_consequence_to_consequence(tc: hl.expr.StructExpression) -> hl.expr.StructExpression:

    """
    Add most_severe_consequence annotation to transcript consequences
    This is for a given transcript, as there are often multiple annotations for a single transcript:
    e.g. splice_region_variant&intron_variant -> splice_region_variant
    """

    csqs = hl.literal(CSQ_ORDER)

    return tc.annotate(
        most_severe_consequence=csqs.find(lambda c: tc.consequence_terms.contains(c))
    )

def process_consequences(mt: Union[hl.MatrixTable, hl.Table], vep_root: str = 'vep',
                         penalize_flags: bool = True) -> Union[hl.MatrixTable, hl.Table]:
    """
    Adds most_severe_consequence (worst consequence for a transcript) into [vep_root].transcript_consequences,
    and worst_csq_by_gene, any_lof into [vep_root]
    :param MatrixTable mt: Input MT
    :param str vep_root: Root for vep annotation (probably vep)
    :param bool penalize_flags: Whether to penalize LOFTEE flagged variants, or treat them as equal to HC
    :return: MT with better formatted consequences
    :rtype: MatrixTable
    """
    csqs = hl.literal(CSQ_ORDER)
    csq_dict = hl.literal(dict(zip(CSQ_ORDER, range(len(CSQ_ORDER)))))

    def find_worst_transcript_consequence(tcl: hl.expr.ArrayExpression) -> hl.expr.StructExpression:
        """
        Gets worst transcript_consequence from an array of em
        """
        flag_score = 500
        no_flag_score = flag_score * (1 + penalize_flags)

        def csq_score(tc):
            return csq_dict[csqs.find(lambda x: x == tc.most_severe_consequence)]
        tcl = tcl.map(lambda tc: tc.annotate(
            csq_score=hl.case(missing_false=True)
            .when((tc.lof == 'HC') & (tc.lof_flags == ''), csq_score(tc) - no_flag_score)
            .when((tc.lof == 'HC') & (tc.lof_flags != ''), csq_score(tc) - flag_score)
            .when(tc.lof == 'LC', csq_score(tc) - 10)
            .when(tc.polyphen_prediction == 'probably_damaging', csq_score(tc) - 0.5)
            .when(tc.polyphen_prediction == 'possibly_damaging', csq_score(tc) - 0.25)
            .when(tc.polyphen_prediction == 'benign', csq_score(tc) - 0.1)
            .default(csq_score(tc))
        ))
        return hl.or_missing(hl.len(tcl) > 0, hl.sorted(tcl, lambda x: x.csq_score)[0])

    transcript_csqs = mt[vep_root].transcript_consequences.map(add_most_severe_consequence_to_consequence)

    gene_dict = transcript_csqs.group_by(lambda tc: tc.gene_symbol)
    worst_csq_gene = gene_dict.map_values(find_worst_transcript_consequence).values()
    sorted_scores = hl.sorted(worst_csq_gene, key=lambda tc: tc.csq_score)
    lowest_score = hl.or_missing(hl.len(sorted_scores) > 0, sorted_scores[0].csq_score)
    gene_with_worst_csq = sorted_scores.filter(lambda tc: tc.csq_score == lowest_score).map(lambda tc: tc.gene_symbol)
    ensg_with_worst_csq = sorted_scores.filter(lambda tc: tc.csq_score == lowest_score).map(lambda tc: tc.gene_id)

    vep_data = mt[vep_root].annotate(transcript_consequences=transcript_csqs,
                                     worst_consequence_term=csqs.find(lambda c: transcript_csqs.map(lambda csq: csq.most_severe_consequence).contains(c)),
                                     worst_csq_by_gene=worst_csq_gene,
                                     any_lof=hl.any(lambda x: x.lof == 'HC', worst_csq_gene),
                                     gene_with_most_severe_csq=gene_with_worst_csq,
                                     ensg_with_most_severe_csq=ensg_with_worst_csq)

    return mt.annotate_rows(**{vep_root: vep_data}) if isinstance(mt, hl.MatrixTable) else mt.annotate(**{vep_root: vep_data})


vds = hl.read_matrix_table('gs://rec_project/genotype_annotations.mt')
#vds.rows().show()
vds = vds.key_rows_by('locus','alleles')
vds = hl.variant_qc(vds)
vds_kon = filter_vep_to_canonical_transcripts(vds)
vds_kon = process_consequences(vds_kon)
vds_kon.describe()
vds_kon.rows().show()
#vds_kon_tb = vds_kon.filter_rows(vds_kon['locus'].contig == 'chr22').rows().select('vep').flatten()
#vds_kon_tb.select('locus','alleles','vep.variant_class','vep.worst_consequence_term','vep.any_lof','vep.gene_with_most_severe_csq','vep.ensg_with_most_severe_csq').export('gs://ddd-edu/topmed/chr_22_vep_konrad.tsv.gz')

In [ ]:
# write intermediate file (above script takes ~20 minutes)
# vds_kon.write('gs://rec_project/vds_kon.mt', overwrite=True)

In [ ]:
vds_kon.count()

In [3]:
vds_kon = hl.read_matrix_table('gs://rec_project/vds_kon.mt')

In [ ]:
vds_kon.describe()

### Explode lines that have more than 1 gene (variants that are in more than one gene).

In [4]:
exploded = vds_kon.explode_rows(vds_kon.vep.gene_with_most_severe_csq)

In [5]:
exploded.rows().show(n=10)

+---------------+------------+---------------+-------------+-------------------+
| locus         | alleles    | rsid          | cm_position | vep.assembly_name |
+---------------+------------+---------------+-------------+-------------------+
| locus<GRCh37> | array<str> | str           |     float64 | str               |
+---------------+------------+---------------+-------------+-------------------+
| 1:723307      | ["C","G"]  | "rs28659788"  |    0.00e+00 | "GRCh37"          |
| 1:727841      | ["G","A"]  | "rs116587930" |    0.00e+00 | "GRCh37"          |
| 1:752721      | ["A","G"]  | "rs3131972"   |    0.00e+00 | "GRCh37"          |
| 1:754105      | ["C","T"]  | "rs12184325"  |    0.00e+00 | "GRCh37"          |
| 1:756604      | ["A","G"]  | "rs3131962"   |    0.00e+00 | "GRCh37"          |
| 1:759036      | ["G","A"]  | "rs114525117" |    0.00e+00 | "GRCh37"          |
| 1:761147      | ["T","C"]  | "rs3115850"   |    0.00e+00 | "GRCh37"          |
| 1:767096      | ["A","G"] 

### Filter genotyped matrix (vds_kon) based on Counsyl recessive gene list. 

In [6]:
with hl.hadoop_open('gs://rec_project/counsyl_recessive_gene_list.txt', 'r') as x:
    counsyl_genes = [line.strip() for line in x]

#### Create new global field for the set of genes

In [7]:
mt = exploded.annotate_globals(counsyl_genes=hl.set(counsyl_genes))

#### Remove rows that do not contain genes in newly created gene field

In [ ]:
mt.describe()

In [8]:
mt_no_genes_filt = mt.filter_rows(mt.counsyl_genes.contains(mt.vep.gene_with_most_severe_csq))

In [9]:
mt_no_genes_filt.count()

(7877, 488295)

In [ ]:
mt_no_genes_filt.rows().show()



### Filter based on frequency: keep variants with frequency < 5% and > 95%

In [10]:
mt_freq_filter = mt_no_genes_filt.filter_rows((mt_no_genes_filt.variant_qc.AF[1] <= 0.05) | (mt_no_genes_filt.variant_qc.AF[1] >= 0.95), keep=True) 

In [11]:
mt_freq_filter.count()

(5616, 488295)

### Filter based on vep.most_severe_consequence

In [12]:
conseq_filter = mt_freq_filter.filter_rows((mt_freq_filter.vep.most_severe_consequence == 'transcript_ablation') | (mt_freq_filter.vep.most_severe_consequence == 'splice_acceptor_variant') | (mt_freq_filter.vep.most_severe_consequence == 'splice_donor_variant') | (mt_freq_filter.vep.most_severe_consequence == 'stop_gained') | (mt_freq_filter.vep.most_severe_consequence == 'frameshift_variant') | (mt_freq_filter.vep.most_severe_consequence == 'start_lost') | (mt_freq_filter.vep.most_severe_consequence == 'initiator_codon_variant') | (mt_freq_filter.vep.most_severe_consequence == 'transcript_amplification') | (mt_freq_filter.vep.most_severe_consequence == 'inframe_insertion') | (mt_freq_filter.vep.most_severe_consequence == 'transcript_deletion') | (mt_freq_filter.vep.most_severe_consequence == 'missense_variant') | (mt_freq_filter.vep.most_severe_consequence == 'protein_altering_variant') | (mt_freq_filter.vep.most_severe_consequence == 'splice_region_variant'), keep=True)

In [13]:
conseq_filter.count()

(3850, 488295)

In [14]:
conseq_filter.write('gs://rec_project/filt_genotype_data.mt', overwrite=True)

2018-12-06 16:09:15 Hail: INFO: wrote matrix table with 3850 rows and 488295 columns in 749 partitions to gs://rec_project/filt_genotype_data.mt


In [15]:
conseq_filter = hl.read_matrix_table('gs://rec_project/filt_genotype_data.mt')

### Filter down to EUR individuals

In [ ]:
keep_table = hl.import_table('gs://rec_project/ukb31063.gwas_samples.both_sexes.txt', 
                             no_header=True, 
                             min_partitions=50)
keep_table = keep_table.rename({'f0': 'Sample'}).key_by('Sample')

mt_subset = conseq_filter.filter_cols(hl.is_defined(keep_table[conseq_filter.s]))
print (mt_subset.count())

### Re-run variant QC

In [ ]:
var_qc = hl.variant_qc(mt_subset.repartition(256)).persist()

### Filter on allele count (AC)

In [ ]:
mt_ac = var_qc.filter_rows((var_qc.variant_qc.AC[0] > 0) & (var_qc.variant_qc.AC[1] > 0), keep=True)
print (mt_ac.count_rows())

### Select desired row and column fields before exporting annotations, export

In [ ]:
genotype_mt_filtered = mt_ac.select_rows(mt_ac.vep.worst_consequence_term, mt_ac.vep.gene_with_most_severe_csq, mt_ac.variant_qc.AC, mt_ac.variant_qc.AF, mt_ac.vep.any_lof, mt_ac.vep.ensg_with_most_severe_csq)
genotype_mt_filtered = genotype_mt_filtered.drop(genotype_mt_filtered.fam_id, genotype_mt_filtered.pat_id, genotype_mt_filtered.mat_id, genotype_mt_filtered.is_female, genotype_mt_filtered.is_case)
# print (genotype_mt_filtered.rows().show())
rows_table = genotype_mt_filtered.rows()
rows_table = rows_table.drop(rows_table.counsyl_genes)
# print (rows_table.count())
rows_table.export('gs://rec_project/annotations_2.tsv.bgz')

In [ ]:
def patched_set_field(self, key, value):
    self._fields[key] = value
    self._fields_inverse[value] = key
    self.__dict__[key] = value
hl.Table._set_field = patched_set_field

### Change GT notation to single digit, select desired entries & rows, export

In [ ]:
genotype_mt_filtered = genotype_mt_filtered.annotate_entries(GTN = genotype_mt_filtered.GT.n_alt_alleles())
genotype_mt_filtered.select_entries('GTN').select_rows().select_cols().make_table().export('gs://rec_project/GT_table_2.tsv.bgz')